-----------------------
#### basic chain
--------------------------

In [1]:
from langchain import PromptTemplate, LLMChain
from langchain_openai  import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

In [2]:
# Define the prompt template
prompt = PromptTemplate.from_template("What is the capital of {country}?")

In [3]:
# Initialize the LLM and chain
llm   = ChatOpenAI()

chain = LLMChain(llm   = llm, 
                 prompt= prompt)

C:\Users\bhupe\AppData\Local\Temp\ipykernel_14080\2838644879.py:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm   = llm,


In [4]:
# Run the chain
result = chain.invoke({"country": "France"})
print(result)  # Output should be "Paris"

{'country': 'France', 'text': 'The capital of France is Paris.'}


example

In [5]:
prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

In [6]:
chain = LLMChain(llm=llm, prompt=prompt)

In [15]:
product = "Nexon"

In [16]:
# Run the chain
result = chain.invoke({"product": product})
print(result)  

{'product': 'Nexon', 'text': 'Nexon Innovations'}


**Simple sequential chain**

In [7]:
from langchain.chains import SimpleSequentialChain

In [8]:
llm_model = 'gpt-3.5-turbo'

In [17]:
llm = ChatOpenAI(temperature=0.9, model=llm_model)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "What is the best name to describe a company that makes {product}?"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [18]:
# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Write a 20 words description for the following company:{company_name}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [19]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two],
                                             verbose=True
                                            )

In [20]:
product = "fitbit"

In [21]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
ActiveLife Technologies
ActiveLife Technologies is a leading provider of innovative health and wellness products, promoting an active lifestyle for improved well-being.

> Finished chain.


{'input': 'fitbit',
 'output': 'ActiveLife Technologies is a leading provider of innovative health and wellness products, promoting an active lifestyle for improved well-being.'}

**Exercise on Simple Sequential chain**

- Text Processing Pipeline

    - Remove punctuation from a text.
    - Convert the text to lowercase.
    - Count the number of words in the text.

In [39]:
# Remove punctuation
prompt_remove_punctuation = PromptTemplate(
    input_variables = ["text"],
    template        = "Remove all punctuation from the following text:\n{text}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=prompt_remove_punctuation)

In [40]:
# Convert to lowercase
prompt_to_lowercase = PromptTemplate(
    input_variables = ["text"],
    template        = "Convert the following text to lowercase:\n{text}"
)

# Chain 2
chain_two = LLMChain(llm=llm, prompt=prompt_to_lowercase)

In [41]:
# Count the words
prompt_count_words = PromptTemplate(
    input_variables =["text"],
    template        = "Count the number of words in the following text and return only the number:\n{text}"
)

# Chain 3
chain_three = LLMChain(llm=llm, prompt=prompt_count_words)

In [42]:
overall_simple_chain = SimpleSequentialChain(chains =[chain_one, chain_two, chain_three],
                                             verbose=True
                                            )

In [43]:
text = 'Modern humans arrived on the Indian subcontinent'

overall_simple_chain.invoke(text)



> Entering new SimpleSequentialChain chain...
Modern humans arrived on the Indian subcontinent
modern humans arrived on the indian subcontinent
7

> Finished chain.


{'input': 'Modern humans arrived on the Indian subcontinent', 'output': '7'}

In [ ]:
overall_simple_chain.